In [1]:
import time
import re

class REPLException(Exception):
    def __init__(self, msg=None):
        Exception.__init__(self, msg)

class REPL(): 
    def wait_prompt(self):
        try: 
            self.receive(".*>>> ", timeout=2, idle_timeout=1)
        except REPLException as e: 
            pass 
        
    def __init__(self, ser):
        self.ser = ser  
        self.wait_prompt() 
        self.ser.write(b'\x03\x02') 
        self.wait_prompt()  
        self.ser.write(b'\x01')
        self.receive("\r\nraw REPL; CTRL-B to exit\r\n>", timeout=2)

    def receive(self, wanted=None, min_length=None, max_length=None, timeout=None, idle_timeout=None): 
        matcher = re.compile(wanted, re.DOTALL) 
        now = time.time()
        if timeout is not None:           timeout_at = now + timeout
        else:                             timeout_at = None

        if idle_timeout is not None: idle_timeout_at = now + idle_timeout
        else:                        idle_timeout_at = None

        buffer = ""
        while True:
            now = time.time()
            ch = self.ser.read(1).decode()
            if len(ch) ==0: ch =None
            if ch is not None:
                buffer += ch
                if idle_timeout is not None: idle_timeout_at = now + idle_timeout

                if matcher is not None and idle_timeout is None and matcher.match(buffer):
                    if min_length is None:          return buffer  
                    elif len(buffer) >= min_length: return buffer
                    
            if idle_timeout_at is not None and now >= idle_timeout_at:
                if matcher.match(buffer): return buffer  
                else: raise REPLException("Did not match at end of idle timeout [%s]" % buffer)

import serial
class MicroBit(): 
    def __init__(self):
        s = serial.Serial(port     = '/dev/ttyACM1', # Change it to 'COM?' in Windows, COM? can be found in Device Manager (Ports)
                          baudrate = 115200, 
                          parity   = serial.PARITY_NONE, 
                          stopbits = serial.STOPBITS_ONE, 
                          timeout  = 0)
        self.repl = REPL(s) 
        
    def send_command(self, cmd):
        self.repl.ser.write(cmd.encode()+b'\x04')  
        self.repl.receive("OK", timeout=1, min_length=2)
        output_text = self.repl.receive(".*\x04")
        self.repl.receive(".*\x04>", timeout=1)  
        return output_text
        
microbit = MicroBit()
microbit.send_command("print(temperature())")

'26\r\n\x04'